# Most popular names in North Carolina by decade
Source: My analysis of data from [NC voter registrations](https://www.ncsbe.gov/results-data/voter-registration-data)

In [ ]:
# # Get the name of the database file
# from pathlib import Path

# # Note: this assumes you have copied the `ncvoters.db` database to this directory
# dbfile = Path("ncvoters.db")

# # # EKH
dbpath = "ncvoters.db" # for some reason it got mad at me when I tried to use the Path, but it was happy with the string

SQL query to be used:

In [ ]:
# query_sql = """
#     SELECT      DISTINCT first_name, COUNT(first_name) as number
#     FROM        voters
#     WHERE       race_code = ?
#     AND         gender_code = ?
#     AND         birth_year BETWEEN ? AND ?
#     GROUP BY    first_name
#     ORDER BY    2 DESC
#     LIMIT       10
# """

# def query(race_code, gender_code):
#     import sqlite3

#     with sqlite3.connect(dbfile) as con:
#         c = con.cursor()
#         for year in range(2000, 1900-1, -10):
#             end_year = year + 10-1
#             c.execute(query_sql, (race_code, gender_code, year, end_year))
#             outrow = [year]
#             for i, row in enumerate(c.fetchall()):
#                 outrow.append(row[0])
#             yield outrow

# # # EKH

import sqlite3
import pandas as pd
import numpy as np

def query(race_code, gender_code, start_year = 1900, end_year = 2000, top = 10):
    
    decadify = lambda x: int(10 * np.floor(x / 10))
    
    assert (start_year == decadify(start_year)) & (end_year == decadify(end_year)), "Make sure your year ends in a zero"
    
    # okay, there's got to be a more elegant way for me to have done this instead of doing multiple nested subqueries
    # but this is what I ended up with
    # (the problem was getting it to recognize 'rank')
    q = f'''
    select *
    from (
        select
            decade,
            first_name,
            row_number() over(partition by decade order by n_rows desc) rank
        from (
            select
                10 * cast(birth_year / 10 as int) decade,
                first_name,
                count(first_name) n_rows            
            from voters
            where race_code = '{race_code}'
            and gender_code = '{gender_code}'
            and birth_year between {start_year} and {end_year}
            group by decade, first_name
        ) a
    ) b
    where rank <= {top}
    '''
    
    # pandas has a sql query reader built right in!
    # I think the pivoting here can also be done in SQL, but I've never gotten the hang of it.
    # much easier to pivot in pandas imho
    with sqlite3.connect(dbpath) as con:
        df = (
            pd.read_sql(q, con)
            .pivot(
                index = "decade",
                columns = "rank",
                values = "first_name"
            )
            .sort_index(ascending = False)
        )
        
    df.columns.name = None # this is just to make the dataframe look nice, it's not necessary
    df.reset_index(inplace = True)
    
    return df

In [ ]:
x = query("W","M")

In [ ]:
x

### White men

In [ ]:
for row in query("W", "M"):
    print(row)